In [1]:
# Connect to MongoDB and create a database for this assignment.
import pymongo
client = pymongo.MongoClient('localhost', 27017)
client.drop_database('hw1') # just to make sure we start from scratch
db = client.hw1

In [2]:
# load the data from modelcollection.json and papercollection.json into Python
import json
with open('modelcollection.json') as model_data:
    model_json = json.load(model_data)
with open('papercollection.json') as paper_data:
    paper_json = json.load(paper_data)

In [3]:
# Put the loaded data into two collections in your database. I recommend calling them models and papers.
for item in model_json:
    db.models.insert_one(item)
for item in paper_json:
    db.papers.insert_one(item)

In [4]:
# How many models are there?
db.models.count()

1114

In [5]:
# What are the field names (keys) for the model entry with _id = 87284?
print(db.models.find_one({"_id":87284}).keys())

dict_keys(['simenvironment', 'transmitters', 'genes', 'channels', 'brainregions', 'text', 'celltypes', '_id', 'modelconcepts', 'receptors', 'references', 'title', 'modeltype'])


In [6]:
# How many distinct cell types are in the models collection?
len(db.models.find().distinct("celltypes"))

188

In [7]:
# Find the list of model ids for models that contain a Hippocampus CA3 pyramidal cell.
cursor = db.models.find({"celltypes":"Hippocampus CA3 pyramidal cell"})
for result_object in cursor:
    print(result_object['_id'])

101629
114337
118098
120907
126814
129067
135902
135903
137259
137505
138421
139421
142104
143148
146499
147756
147867
148035
150288
151282
168314
168874
181967
184139
185512
186768
189088
20007
3263
35358
7907
84606
87216
87762
98003


In [8]:
# What other cells appear in models with a Hippocampus CA3 pyramidal cell? 
# Sort them in alphabetical order. How many such cells are there?
cursor = db.models.find({"celltypes":"Hippocampus CA3 pyramidal cell"})
res = set()
for result_object in cursor:
    for celltype in result_object['celltypes']:
        res.add(celltype)
res.remove("Hippocampus CA3 pyramidal cell")
print(sorted(res))
print(len(res))

['Abstract Izhikevich neuron', 'Abstract integrate-and-fire adaptive exponential (AdEx) neuron', 'Cerebellum purkinje cell', 'Dentate gyrus basket cell', 'Dentate gyrus granule cell', 'Dentate gyrus hilar cell', 'Dentate gyrus mossy cell', 'Entorhinal cortex stellate cell', 'Hippocampus CA1 axo-axonic cell', 'Hippocampus CA1 basket cell', 'Hippocampus CA1 bistratified cell', 'Hippocampus CA1 interneuron oriens alveus', 'Hippocampus CA1 pyramidal cell', 'Hippocampus CA3 axo-axonic cells', 'Hippocampus CA3 basket cell', 'Hippocampus CA3 stratum oriens lacunosum-moleculare interneuron', 'Hippocampus septum medial GABAergic neuron', 'Hodgkin-Huxley neuron', 'Neocortex fast spiking (FS) interneuron', 'Neocortex layer 2-3 pyramidal cell', 'Neocortex layer 4 pyramidal cell', 'Neocortex layer 5-6 pyramidal cell', 'Neocortex spiking regular (RS) neuron', 'Neocortical pyramidal cortical-thalamic cell', 'Pinsky-Rinzel CA1/3 pyramidal cell ']
25


In [19]:
# How many models are there for each cell type? 
# Display the results in a formatted table,
# sorted from most commonly appearing cell type to least commonly appearing.
list(db.models.aggregate([
        {
            '$unwind': '$celltypes'
        },
        {
            '$group': 
            {
                '_id': '$celltypes',
                'count': {'$sum': 1}
            },
        },
        {
            '$sort': {'count': -1}
        }
    ]))

[{'_id': 'Neocortex layer 5-6 pyramidal cell', 'count': 108},
 {'_id': 'Hippocampus CA1 pyramidal cell', 'count': 104},
 {'_id': 'Neocortex layer 2-3 pyramidal cell', 'count': 60},
 {'_id': 'Hippocampus CA3 pyramidal cell', 'count': 35},
 {'_id': 'Neocortex fast spiking (FS) interneuron', 'count': 30},
 {'_id': 'Olfactory bulb main mitral cell', 'count': 30},
 {'_id': 'Hodgkin-Huxley neuron', 'count': 29},
 {'_id': 'Thalamus geniculate nucleus (lateral) principal neuron',
  'count': 26},
 {'_id': 'Abstract integrate-and-fire leaky neuron', 'count': 25},
 {'_id': 'Cerebellum purkinje cell', 'count': 24},
 {'_id': 'Dentate gyrus granule cell', 'count': 24},
 {'_id': 'Neocortex spiking regular (RS) neuron', 'count': 22},
 {'_id': 'Neostriatum spiny direct pathway neuron', 'count': 22},
 {'_id': 'Neocortex spiking low threshold (LTS) neuron', 'count': 21},
 {'_id': 'Neocortex layer 4 pyramidal cell', 'count': 20},
 {'_id': 'Olfactory bulb main interneuron granule MC cell', 'count': 19},
 {

In [33]:
# Find the model titles (not paper titles) for models that 
# (1) involve a Hippocampus CA3 pyramidal cell, and 
# (2) have an associated reference where one of the authors is "Migliore M".
for model in db.models.aggregate([
        {
           '$unwind': '$references'
        },
         
        { 
            '$lookup':
            {
                'from': 'papers',
                'localField': 'references',
                'foreignField': '_id',
                'as': 'papers'
            }
        },
        {
            '$match':
            {
                'papers.authors': 'Migliore M'
            }
        }
    ]):
    print(model['title'])

CA3 pyramidal neuron: firing properties (Hemond et al. 2008)
CA1 pyramidal neuron: calculation of MRI signals (Cassara et al. 2008)
Nigral dopaminergic neurons: effects of ethanol on Ih (Migliore et al. 2008)
CA1 pyramidal neuron: functional significance of axonal Kv7 channels (Shah et al. 2008)
Olfactory bulb mitral and granule cell column formation (Migliore et al. 2007)
CA1 pyramidal neuron: h channel-dependent deficit of theta oscill. resonance (Marcelin et al. 2008)
Ca3 pyramidal neuron: membrane response near rest (Hemond et al. 2009)
CA1 pyramidal neurons: effects of a Kv7.2 mutation (Miceli et al. 2009)
Computational neuropharmacology of CA1 pyramidal neuron (Ferrante et al. 2008)
Dentate Gyrus Feed-forward inhibition (Ferrante et al. 2009)
NAcc medium spiny neuron: effects of cannabinoid withdrawal (Spiga et al. 2010)
CA1 pyramidal neuron: rebound spiking (Ascoli et al.2010)
CA3 pyramidal neuron (Safiulina et al. 2010)
Olfactory bulb cluster formation (Migliore et al. 2010)
A 

In [51]:
# Find all the authors who were on a paper associated with
# a model that involved a Hippocampus CA3 pyramidal cell. 
# Sort them in alphabetical order; 
# Give this list and state its length.
for papaer in db.papers.aggregate([
        {
           '$unwind': '$_id'
        },
         
        { 
            '$lookup':
            {
                'from': 'models',
                'localField': '_id',
                'foreignField': 'references',
                'as': 'models'
            }
        },
        {
            '$match':
            {
                'celltypes': 'Hippocampus CA3 pyramidal cell'
            }
        }
    ]):
    print(papaer.keys())

In [56]:
# Rename the Hippocampus CA1 pyramidal cell to be the Hippocampus CA1 pyramidal neuron. 
# (Note: here we're using CA1 instead of CA3.) 
# Make sure that this is consistent across all documents in the models collection.
db.papers.update_many({'celltypes': {'$regex': 'Hippocampus CA1 pyramidal cell'}},
                      {'$set': {'celltypes': 'Hippocampus CA3 pyramidal cell'}})

In [59]:
# Add a new entry (make up the data, but keep it appropriate) to the models collection. 
db.papers.insert_one({
                  'last modified': 'Jan 1st 2017',
                  'channels': ['I Na,t', 'I Na,p', 'I Potassium', 'I A', 'I K', 'I M',
                               'I L high threshold', 'I N', 'I T low threshold', 'I p,q',
                               'I K,Ca', 'I h'],
                  'transmitters': ['NO', 'Glutamate'],
                  'average dendrite length': 4586
              })

In [58]:
db.models.find({})